In [25]:
import numpy as np
import torch
import torch.nn as nn

import sys
sys.path.insert(0, '../ml')
from customlayers import *

In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
params = {}

hparams = {}

#Training data
hparams['N_samples'] = 10000
hparams['d'] = 2
hparams['l_min'] = 0.5
hparams['l_max'] = 1

#Training settings
hparams['dtype'] = torch.float64
hparams['precision'] = 64
hparams['devices'] = [0]
hparams['matrix_loss'] = None #relativefrobeniusnorm
hparams['optimizer1'] = torch.optim.Adam
hparams['optimizer2'] = torch.optim.LBFGS
hparams['switch_threshold'] = 5e-3
hparams['learning_rate'] = 1e-3
hparams['batch_size'] = 100
hparams['epochs'] = 5000

#System net
hparams['modeltype'] = 'NGO'
hparams['model/data'] = 'model'
hparams['gamma_stabilization'] = 0
hparams['kernel_sizes'] = [5,5,2,2,2,2,5,5]
hparams['N_w'] = 30000
hparams['bottleneck_size'] = 20
hparams['NLB_outputactivation'] = nn.Tanhshrink()
hparams['bias_NLBranch'] = False
hparams['bias_LBranch'] = False

#Bases
hparams['h'] = [10,10]
hparams['p'] = 3
hparams['C'] = 2
hparams['N'] = np.prod(hparams['h'])

#Quadrature
hparams['quadrature'] = 'Gauss-Legendre'
hparams['n_elements'] = max(int((hparams['h'][0] - 1)/hparams['p']), 1)
hparams['Q'] = 33*hparams['n_elements']

hparams['quadrature_L'] = 'Gauss-Legendre'
hparams['n_elements_L'] = max(int((hparams['h'][0] - 1)/hparams['p']), 1)
hparams['Q_L'] = 33*hparams['n_elements_L']

#Symmetries
hparams['scaling_equivariance'] = False
hparams['permutation_equivariance'] = False

params['hparams'] = hparams

In [89]:
class InvCNN(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.hparams = params['hparams']
        #Balancing the number of trainable parameters to N_w
        self.num_channels = 1
        count = 0
        num_channels_list = []
        while count < self.hparams['N_w']:
            self.init_layers()
            count = sum(p.numel() for p in self.parameters())
            num_channels_list.append(self.num_channels)
            self.num_channels +=1
        self.num_channels = num_channels_list[-2]
        self.init_layers()
        
    def init_layers(self):
        self.layers = nn.ModuleList()
        self.kernel_sizes = self.hparams['kernel_sizes']
        self.bottleneck_size = self.hparams['bottleneck_size']
        #Layers
        self.layers.append(ReshapeLayer(output_shape=(1,self.hparams['h'][0],self.hparams['h'][1])) if self.hparams['model/data']=='data' else ReshapeLayer(output_shape=(1,self.hparams['N'],self.hparams['N'])))
        self.layers.append(nn.Conv2d(in_channels=1, out_channels=self.num_channels, kernel_size=self.kernel_sizes[0], stride=self.kernel_sizes[0], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.LeakyReLU())
        self.layers.append(nn.Conv2d(in_channels=self.num_channels, out_channels=self.num_channels, kernel_size=self.kernel_sizes[1], stride=self.kernel_sizes[1], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.LeakyReLU())
        self.layers.append(nn.Conv2d(in_channels=self.num_channels, out_channels=self.num_channels, kernel_size=self.kernel_sizes[2], stride=self.kernel_sizes[2], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.LeakyReLU())
        self.layers.append(nn.Conv2d(in_channels=self.num_channels, out_channels=self.bottleneck_size, kernel_size=self.kernel_sizes[3], stride=self.kernel_sizes[3], bias=self.hparams.get('bias_NLBranch', True)))
        # self.layers.append(ReshapeLayer(output_shape=(int(self.bottleneck_size),)))
        self.layers.append(InversionLayer())
        self.layers.append(nn.LeakyReLU())
        # self.layers.append(ReshapeLayer(output_shape=(self.bottleneck_size,1,1)))
        self.layers.append(nn.ConvTranspose2d(self.bottleneck_size, self.num_channels, kernel_size=self.kernel_sizes[4], stride=self.kernel_sizes[4], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.LeakyReLU())
        self.layers.append(nn.ConvTranspose2d(self.num_channels, self.num_channels, kernel_size=self.kernel_sizes[5], stride=self.kernel_sizes[5], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.LeakyReLU())
        self.layers.append(nn.ConvTranspose2d(self.num_channels, self.num_channels, kernel_size=self.kernel_sizes[6], stride=self.kernel_sizes[6], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.LeakyReLU())
        self.layers.append(nn.ConvTranspose2d(self.num_channels, 1, kernel_size=self.kernel_sizes[7], stride=self.kernel_sizes[7], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(ReshapeLayer(output_shape=(self.hparams['N'],self.hparams['N'])))
        if self.hparams['NLB_outputactivation'] is not None:
            self.layers.append(self.hparams['NLB_outputactivation'])

    def forward(self, x):
        if self.hparams.get('permutation_equivariance',False)==True:
            x, row_sorted_indices, col_sorted_indices = sort_matrices(x)
        if self.hparams.get('scaling_equivariance',False)==True:
            x_norm = torch.amax(torch.abs(x), dim=(-1,-2))
            x = x/x_norm[:,None,None]
        for layer in self.layers:
            x = layer(x)
        y = x
        if self.hparams.get('scaling_equivariance',False)==True:
            y = y/x_norm[:,None,None]    
        if self.hparams.get('permutation_equivariance',False)==True:
            y = unsort_matrices(y, row_sorted_indices, col_sorted_indices)
        return y

In [ ]:
class PrelongationCNN(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.hparams = params['hparams']
        #Balancing the number of trainable parameters to N_w
        self.num_channels = 1
        count = 0
        num_channels_list = []
        while count < self.hparams['N_w']:
            self.init_layers()
            count = sum(p.numel() for p in self.parameters())
            num_channels_list.append(self.num_channels)
            self.num_channels +=1
        self.num_channels = num_channels_list[-2]
        self.init_layers()
        
    def init_layers(self):
        self.layers = nn.ModuleList()
        self.kernel_sizes = self.hparams['kernel_sizes']
        self.bottleneck_size = self.hparams['bottleneck_size']
        #Layers
        self.layers.append(ReshapeLayer(output_shape=(1,self.hparams['h_F'][0],self.hparams['h_F'][1])) if self.hparams['model/data']=='data' else ReshapeLayer(output_shape=(1,self.hparams['N_F'],self.hparams['N_F'])))
        self.layers.append(nn.ConvTranspose2d(1, self.num_channels, kernel_size=self.kernel_sizes[4], stride=self.kernel_sizes[4], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.LeakyReLU())
        self.layers.append(nn.ConvTranspose2d(self.num_channels, self.num_channels, kernel_size=self.kernel_sizes[5], stride=self.kernel_sizes[5], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.LeakyReLU())
        self.layers.append(nn.ConvTranspose2d(self.num_channels, self.num_channels, kernel_size=self.kernel_sizes[6], stride=self.kernel_sizes[6], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.LeakyReLU())
        self.layers.append(nn.ConvTranspose2d(self.num_channels, 1, kernel_size=self.kernel_sizes[7], stride=self.kernel_sizes[7], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(ReshapeLayer(output_shape=(self.hparams['N'],self.hparams['N'])))
        if self.hparams['NLB_outputactivation'] is not None:
            self.layers.append(self.hparams['NLB_outputactivation'])

    def forward(self, x):
        if self.hparams.get('permutation_equivariance',False)==True:
            x, row_sorted_indices, col_sorted_indices = sort_matrices(x)
        if self.hparams.get('scaling_equivariance',False)==True:
            x_norm = torch.amax(torch.abs(x), dim=(-1,-2))
            x = x/x_norm[:,None,None]
        for layer in self.layers:
            x = layer(x)
        y = x
        if self.hparams.get('scaling_equivariance',False)==True:
            y = y/x_norm[:,None,None]    
        if self.hparams.get('permutation_equivariance',False)==True:
            y = unsort_matrices(y, row_sorted_indices, col_sorted_indices)
        return y

In [90]:
x = torch.rand(hparams['batch_size'],hparams['N'],hparams['N'])
x.shape

torch.Size([100, 100, 100])

In [91]:
model = InvCNN(params)

In [92]:
sum(p.numel() for p in model.parameters())

29988

In [93]:
model.forward(x)

tensor([[[-7.4900e-03,  1.2789e-04, -4.3645e-04,  ...,  1.2737e-02,
          -1.5404e-06,  1.3446e-02],
         [ 4.7337e-04,  1.6133e-04, -8.3207e-04,  ...,  3.7509e-03,
           1.8909e-01,  7.3698e-04],
         [ 1.7584e-04,  1.0041e-02,  9.7789e-06,  ...,  2.2210e-01,
           6.4823e-03, -6.4596e-04],
         ...,
         [-1.6802e-03,  4.8157e-02,  6.0311e-03,  ...,  1.7680e-04,
           5.7013e-02, -1.8016e-02],
         [-3.9019e-05, -8.8141e-03, -3.3028e-05,  ...,  5.7068e-05,
           8.9847e-02,  1.1832e-02],
         [-4.0374e-04, -2.7474e-08,  1.1369e-02,  ..., -1.9241e-05,
          -3.3357e-02, -3.0098e-02]],

        [[-2.1715e-04, -2.2836e-05, -2.8033e-06,  ...,  9.9324e-04,
           4.4893e-05,  1.1036e-07],
         [ 0.0000e+00,  2.6911e-05, -6.3397e-05,  ...,  1.3068e-04,
           4.8744e-03,  5.7295e-06],
         [-1.0477e-09,  8.4996e-04,  4.4085e-05,  ...,  5.8231e-03,
           1.0273e-04,  2.5611e-09],
         ...,
         [-6.0722e-07,  7